# 转置卷积



In [1]:
import torch
from torch import nn
from d2l import torch as d2l

实现基本的转置卷积运算

输入 X:<br>
[1 2]<br>
[3 4]<br>
卷积核 K:<br>
[1 0]<br>
[0 1]<br>
输出 Y (3×3):<br>
[1 2 0]<br>
[3 4 2]<br>
[0 3 4]

计算过程：<br>
X[0,0]=1 → 在Y的(0,0)位置叠加 1*K<br>
X[0,1]=2 → 在Y的(0,1)位置叠加 2*K<br>
X[1,0]=3 → 在Y的(1,0)位置叠加 3*K<br>
X[1,1]=4 → 在Y的(1,1)位置叠加 4*K

| 部分                      | 含义                                        |
| ----------------------- | ----------------------------------------- |
| `X[i, j]`               | 输入矩阵中的一个**标量值**（单个数字）                     |
| `X[i, j] * K`           | 把这个数字乘以卷积核 `K` 的**每个元素**（广播机制）            |
| `Y[i: i + h, j: j + w]` | 输出矩阵 `Y` 中，从 `(i, j)` 开始、大小为 `h × w` 的子区域 |
| `+=`                    | **累加**到该子区域（不是覆盖，是相加）                     |


In [2]:
def trans_conv(X, K):
    h, w = K.shape # 1. 获取卷积核的高度和宽度
    # 2. 初始化输出矩阵
    # 输出尺寸 = 输入尺寸 + 卷积核尺寸 - 1
    Y = torch.zeros((X.shape[0] + h - 1, X.shape[1] + w - 1))
    for i in range(X.shape[0]):
        for j in range(X.shape[1]):
            # 4. 将输入元素与整个卷积核相乘，加到输出的对应区域
            Y[i: i + h, j: j + w] += X[i, j] * K
    return Y

验证上述实现输出

![](./img/转置卷积1.png)

In [3]:
X = torch.tensor([[0.0, 1.0], [2.0, 3.0]])
K = torch.tensor([[0.0, 1.0], [2.0, 3.0]])
trans_conv(X, K)

tensor([[ 0.,  0.,  1.],
        [ 0.,  4.,  6.],
        [ 4., 12.,  9.]])

使用高级API获得相同的结果

In [4]:
X, K = X.reshape(1, 1, 2, 2), K.reshape(1, 1, 2, 2) # (batch, channel, height, width)
# 第1个参数 1：输入通道数；第2个参数 1：输出通道数；
# kernel_size=2：卷积核大小为 2×2；bias=False：不使用偏置项（与手动实现一致）
tconv = nn.ConvTranspose2d(1, 1, kernel_size=2, bias=False)
# 将自定义卷积核 K 赋值给转置卷积层的权重参数。，这里K是之前定义的2×2卷积核矩阵。
tconv.weight.data = K
# 输出尺寸 = 输入尺寸 + 卷积核尺寸 - 1 = 2 + 2 - 1 = 3
tconv(X)

tensor([[[[ 0.,  0.,  1.],
          [ 0.,  4.,  6.],
          [ 4., 12.,  9.]]]], grad_fn=<ConvolutionBackward0>)

填充、步幅和多通道

具体运算过程<br>
先计算无padding的完整输出（3×3）：<br>
[a b c]<br>
[d e f]<br>
[g h i]<br>
这个中间结果与你之前手动实现的 trans_conv(X, K) 完全相同。<br>
再裁剪边缘：padding=1 表示从每边各裁剪1个像素，只保留中心区域：<br>
[e]  ← 只保留这个中心元素

In [5]:
# 在转置卷积中，padding 的作用是：裁剪输出边缘，而不是填充输入。
# 计算公式：输出尺寸 = 输入尺寸 + 卷积核尺寸 - 1 - 2 × padding
tconv = nn.ConvTranspose2d(1, 1, kernel_size=2, padding=1, bias=False)
tconv.weight.data = K
tconv(X)

tensor([[[[4.]]]], grad_fn=<ConvolutionBackward0>)

| 输入元素       | 发射的块                  | 在输出中的位置         | 贡献值             |
| ---------- | --------------------- | --------------- | --------------- |
| `X[0,0]=0` | `0*K`                 | 左上角 (0,0)-(1,1) | 全0              |
| `X[0,1]=1` | `1*K = [[0,1],[2,3]]` | 右上角 (0,2)-(1,3) | `[[0,1],[2,3]]` |
| `X[1,0]=2` | `2*K = [[0,2],[4,6]]` | 左下角 (2,0)-(3,1) | `[[0,2],[4,6]]` |
| `X[1,1]=3` | `3*K = [[0,3],[6,9]]` | 右下角 (2,2)-(3,3) | `[[0,3],[6,9]]` |


![](./img/转置卷积2.png)

In [6]:
# 当 stride=2 时，转置卷积会上采样（放大）输入特征图，这是转置卷积最常见的用途。
# 输出尺寸公式:输出尺寸 = (输入尺寸 - 1) × stride + 卷积核尺寸 - 2 × padding
tconv = nn.ConvTranspose2d(1, 1, kernel_size=2, stride=2, bias=False)
tconv.weight.data = K
tconv(X)

tensor([[[[0., 0., 0., 1.],
          [0., 0., 2., 3.],
          [0., 2., 0., 3.],
          [4., 6., 6., 9.]]]], grad_fn=<ConvolutionBackward0>)

In [7]:
X = torch.rand(size=(1, 10, 16, 16)) # (batch, channels, height, width)
# 输出尺寸公式：output = floor((input + 2*padding - kernel) / stride + 1)
conv = nn.Conv2d(10, 20, kernel_size=5, padding=2, stride=3)
# 输出尺寸公式：output = (input - 1) * stride - 2*padding + kernel
tconv = nn.ConvTranspose2d(20, 10, kernel_size=5, padding=2, stride=3)
tconv(conv(X)).shape == X.shape

True

与矩阵变换的联系

Y[i,j] = sum( X[i:i+h, j:j+w] * K )其中 h=2, w=2 是卷积核的大小。

✅ corr2d = 深度学习中的"卷积"（实际数学卷积需要翻转核，但深度学习中通常不翻转）<br>
✅ 无填充、步长为1时：输出尺寸 = 输入尺寸 - 卷积核尺寸 + 1<br>
✅ 运算本质是：卷积核K在X上滑动，逐元素相乘再求和（点积）

1. 计算 Y[0,0]<br>
X[0:2, 0:2] = [[0., 1.],[3., 4.]]
Y[0,0] = 0*1 + 1*2 + 3*3 + 4*4 = 0 + 2 + 9 + 16 = 27<br>
2. 计算 Y[0,1]<br>
X[0:2, 1:3] = [[1., 2.],[4., 5.]]
Y[0,1] = 1*1 + 2*2 + 4*3 + 5*4 = 1 + 4 + 12 + 20 = 37<br>
3. 计算 Y[1,0]<br>
X[1:3, 0:2] = [[3., 4.],[6., 7.]]
Y[1,0] = 3*1 + 4*2 + 6*3 + 7*4 = 3 + 8 + 18 + 28 = 57<br>
4. 计算 Y[1,1]<BR>
X[1:3, 1:3] = [[4., 5.],[7., 8.]]
Y[1,1] = 4*1 + 5*2 + 7*3 + 8*4 = 4 + 10 + 21 + 32 = 67

In [8]:
'''
X = [[0., 1., 2.],
    [3., 4., 5.],
    [6., 7., 8.]]
'''
X = torch.arange(9.0).reshape(3, 3)
K = torch.tensor([[1.0, 2.0], [3.0, 4.0]])
Y = d2l.corr2d(X, K)
Y

tensor([[27., 37.],
        [57., 67.]])

In [9]:
def kernel2matrix(K):
    # 1. 创建长度为5的零向量k，和4×9的零矩阵W
    k, W = torch.zeros(5), torch.zeros((4, 9))
    '''
    2. 将2×2核K的元素填充到k的特殊位置
    k[0]=a, k[1]=b, k[3]=c, k[4]=d
    此时 k = [a, b, 0, c, d]  （中间故意留0）
    '''
    k[:2], k[3:5] = K[0, :], K[1, :]
    '''
    3. 将k以滑动窗口方式填充到W的每一行
    第0行: [a, b, 0, c, d, 0, 0, 0, 0]
    第1行: [0, a, b, 0, c, d, 0, 0, 0]
    第2行: [0, 0, 0, a, b, 0, c, d, 0]
    第3行: [0, 0, 0, 0, a, b, 0, c, d]
    '''
    W[0, :5], W[1, 1:6], W[2, 3:8], W[3, 4:] = k, k, k, k
    return W

W = kernel2matrix(K)
W

tensor([[1., 2., 0., 3., 4., 0., 0., 0., 0.],
        [0., 1., 2., 0., 3., 4., 0., 0., 0.],
        [0., 0., 0., 1., 2., 0., 3., 4., 0.],
        [0., 0., 0., 0., 1., 2., 0., 3., 4.]])

In [10]:
'''
1. X.reshape(-1)——展平输入:将3×3的输入矩阵X展平成9×1的列向量
2. torch.matmul(W, X.reshape(-1))——矩阵乘法
W是4×9矩阵，X是9×1向量，结果是一个4×1向量：W(4×9)×X(9×1)=Temp(4×1)
3. .reshape(2, 2) —— 重塑为卷积输出形状:将4×1向量重新排列成2×2矩阵（与原卷积输出Y同形状）
4. Y == ... —— 逐元素相等性检查:返回一个2×2的布尔张量，验证两种方式结果是否一致
'''
Y == torch.matmul(W, X.reshape(-1)).reshape(2, 2)

tensor([[True, True],
        [True, True]])

In [11]:
'''
1. Z = trans_conv(Y, K)
Y:普通卷积的输出（2×2）;K:2×2卷积核;trans_conv:手动实现的转置卷积函数;Z:结果（3×3），尺寸还原为原始输入X的大小
2. W.T
之前kernel2matrix(K)构造的矩阵W的转置;如果W是4×9（将3×3 → 2×2）;那么W.T就是9×4（将2×2→3×3）
3. torch.matmul(W.T, Y.reshape(-1))
将2×2的Y展平成4×1向量;用9×4的W.T乘4×1的Y→9×1的临时向量;等价于执行了转置卷积的线性变换
4. .reshape(3, 3)
将9×1向量重塑为3×3矩阵，与Z同形状
'''
Z = trans_conv(Y, K)
Z == torch.matmul(W.T, Y.reshape(-1)).reshape(3, 3)

tensor([[True, True, True],
        [True, True, True],
        [True, True, True]])